# 🎬 LOCOMOT.IO Clip Generator Pro

**Two modes for generating promotional clips:**

1. **Discovery Mode** - Runs simulation, ML model learns what's exciting, extracts best moments
2. **Director Mode** - Scripts cinematic scenarios (underdog comeback, kill rampage, etc.)

Features:
- Full game-accurate rendering
- Learned excitement scoring (not hardcoded)
- Strafe detection (rejects coiling)
- Pre-built cinematic scenarios
- Exports as MP4

In [ ]:
!pip install -q numpy pillow imageio imageio-ffmpeg scikit-learn

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import imageio
from collections import deque
import random
import math
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Dict, Callable
from sklearn.ensemble import RandomForestRegressor
from IPython.display import Video, display
import os

In [ ]:
# === CONSTANTS ===
WORLD_W, WORLD_H = 100, 75
GRID = 16
CANVAS_W, CANVAS_H = 640, 480
FPS = 30

TRAIN_COLORS = [
    (255, 0, 255), (0, 170, 255), (255, 85, 85), (255, 200, 0),
    (0, 255, 200), (255, 100, 200), (180, 100, 255), (100, 255, 100),
]

GUN_TYPES = [
    {'name': 'PISTOL', 'color': (150, 150, 150)},
    {'name': 'SMG', 'color': (100, 200, 255)},
    {'name': 'SHOTGUN', 'color': (255, 150, 50)},
    {'name': 'LASER', 'color': (255, 0, 255)},
]

In [ ]:
# === TRAIN CLASS ===
@dataclass
class Segment:
    x: int
    y: int
    hp: int = 100
    max_hp: int = 100
    gun: Optional[Dict] = None

@dataclass 
class Train:
    id: int
    name: str
    segments: List[Segment] = field(default_factory=list)
    color: Tuple[int, int, int] = (255, 0, 255)
    direction: Tuple[int, int] = (1, 0)
    alive: bool = True
    kills: int = 0
    brain: np.ndarray = None
    brain2: np.ndarray = None
    is_protagonist: bool = False  # For director mode
    script: Optional[Callable] = None  # Custom behavior
    direction_history: List[Tuple[int, int]] = field(default_factory=list)
    position_history: List[Tuple[int, int]] = field(default_factory=list)
    
    def __post_init__(self):
        if self.brain is None:
            self.brain = np.random.randn(12, 16) * 0.5
            self.brain2 = np.random.randn(16, 3) * 0.5
    
    @property
    def head(self): return self.segments[0] if self.segments else None
    
    @property
    def length(self): return len(self.segments)
    
    def get_vision(self, game) -> np.ndarray:
        if not self.head: return np.zeros(12)
        vision = []
        dirs = [(0,-1), (1,-1), (1,0), (1,1), (0,1), (-1,1), (-1,0), (-1,-1)]
        for dx, dy in dirs:
            dist, x, y = 0, self.head.x, self.head.y
            while dist < 15:
                x, y = (x + dx) % WORLD_W, (y + dy) % WORLD_H
                dist += 1
                if any(seg.x == x and seg.y == y for t in game.trains if t.id != self.id and t.alive for seg in t.segments):
                    break
            vision.append(1.0 - dist / 15.0)
        if game.food:
            nf = min(game.food, key=lambda f: abs(f[0]-self.head.x) + abs(f[1]-self.head.y))
            vision.extend([(nf[0] - self.head.x) / WORLD_W, (nf[1] - self.head.y) / WORLD_H])
        else:
            vision.extend([0, 0])
        vision.extend([self.direction[0], self.direction[1]])
        return np.array(vision)
    
    def think(self, game) -> int:
        if self.script:
            return self.script(self, game)
        v = self.get_vision(game)
        return int(np.argmax(np.tanh(v @ self.brain) @ self.brain2))
    
    def turn(self, decision: int):
        dx, dy = self.direction
        if decision == 0: self.direction = (dy, -dx)
        elif decision == 2: self.direction = (-dy, dx)
        self.direction_history.append(self.direction)
        if len(self.direction_history) > 60: self.direction_history.pop(0)
    
    def move(self):
        if not self.alive or not self.head: return
        new_x = (self.head.x + self.direction[0]) % WORLD_W
        new_y = (self.head.y + self.direction[1]) % WORLD_H
        self.position_history.append((new_x, new_y))
        if len(self.position_history) > 60: self.position_history.pop(0)
        for i in range(len(self.segments) - 1, 0, -1):
            self.segments[i].x, self.segments[i].y = self.segments[i-1].x, self.segments[i-1].y
        self.head.x, self.head.y = new_x, new_y
    
    def grow(self, n=1):
        for _ in range(n):
            t = self.segments[-1]
            self.segments.append(Segment(t.x, t.y, gun=random.choice(GUN_TYPES) if random.random()<0.3 else None))
    
    def is_strafing(self) -> bool:
        if len(self.position_history) < 30: return True
        s, e = self.position_history[0], self.position_history[-1]
        dx = min(abs(e[0]-s[0]), WORLD_W - abs(e[0]-s[0]))
        dy = min(abs(e[1]-s[1]), WORLD_H - abs(e[1]-s[1]))
        return math.sqrt(dx*dx + dy*dy) > 8

In [ ]:
# === EXCITEMENT MODEL (Learns what's exciting) ===
class ExcitementModel:
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=50, max_depth=8)
        self.training_data = []  # (features, label)
        self.is_trained = False
    
    def extract_features(self, event: Dict, game_state: Dict) -> np.ndarray:
        """Extract features from a moment for excitement prediction"""
        return np.array([
            event.get('kill', 0),                    # Is this a kill?
            event.get('streak', 0),                  # Kill streak count
            event.get('length_diff', 0) / 20,        # Size difference (underdog bonus)
            event.get('velocity', 0) / 10,           # How fast was killer moving?
            event.get('near_miss_count', 0) / 5,     # Close calls in last 2 sec
            event.get('trains_nearby', 0) / 5,       # Action density
            event.get('killer_health_pct', 1.0),     # Low health = more exciting
            event.get('victim_length', 0) / 30,      # Bigger victim = better
            event.get('leader_involved', 0),         # Leader in the action?
            event.get('comeback', 0),                # Was killer losing before?
        ])
    
    def add_sample(self, features: np.ndarray, excitement: float):
        """Add a training sample (auto-labeled or human-labeled)"""
        self.training_data.append((features, excitement))
    
    def auto_label(self, event: Dict) -> float:
        """Heuristic auto-labeling for initial training"""
        score = 0.0
        if event.get('kill'): score += 0.4
        score += event.get('streak', 0) * 0.15
        score += max(0, -event.get('length_diff', 0)) * 0.02  # Underdog bonus
        score += (1 - event.get('killer_health_pct', 1)) * 0.2  # Low health exciting
        score += event.get('comeback', 0) * 0.3
        score += event.get('near_miss_count', 0) * 0.05
        return min(1.0, score)
    
    def train(self):
        """Train the model on collected samples"""
        if len(self.training_data) < 20:
            print(f"  Need more data ({len(self.training_data)}/20 samples)")
            return
        X = np.array([d[0] for d in self.training_data])
        y = np.array([d[1] for d in self.training_data])
        self.model.fit(X, y)
        self.is_trained = True
        print(f"  ✅ Excitement model trained on {len(self.training_data)} samples")
    
    def predict(self, features: np.ndarray) -> float:
        """Predict excitement score (0-1)"""
        if not self.is_trained:
            return self.auto_label({'kill': features[0], 'streak': features[1]*5,
                                   'length_diff': features[2]*20, 'killer_health_pct': features[6],
                                   'comeback': features[9], 'near_miss_count': features[4]*5})
        return float(self.model.predict(features.reshape(1, -1))[0])

In [ ]:
# === DIRECTOR MODE (Scripted Scenarios) ===
class Director:
    """Creates scripted cinematic scenarios"""
    
    @staticmethod
    def underdog_comeback(protagonist: Train, game) -> int:
        """Small train hunts bigger trains aggressively"""
        # Find nearest bigger train
        targets = [t for t in game.trains if t.alive and t.id != protagonist.id and t.length > protagonist.length]
        if not targets:
            targets = [t for t in game.trains if t.alive and t.id != protagonist.id]
        if not targets:
            return 1  # Go straight
        
        target = min(targets, key=lambda t: abs(t.head.x - protagonist.head.x) + abs(t.head.y - protagonist.head.y))
        return Director._move_toward(protagonist, target.head.x, target.head.y)
    
    @staticmethod
    def kill_rampage(protagonist: Train, game) -> int:
        """Aggressively hunt any nearby train"""
        targets = [t for t in game.trains if t.alive and t.id != protagonist.id]
        if not targets:
            return 1
        target = min(targets, key=lambda t: abs(t.head.x - protagonist.head.x) + abs(t.head.y - protagonist.head.y))
        return Director._move_toward(protagonist, target.head.x, target.head.y)
    
    @staticmethod
    def narrow_escape(protagonist: Train, game) -> int:
        """Weave through danger, barely surviving"""
        # Find threats and dodge them narrowly
        threats = []
        for t in game.trains:
            if t.alive and t.id != protagonist.id:
                for seg in t.segments:
                    dist = abs(seg.x - protagonist.head.x) + abs(seg.y - protagonist.head.y)
                    if dist < 8:
                        threats.append((seg.x, seg.y, dist))
        
        if not threats:
            # No immediate threat - find some action
            return Director.kill_rampage(protagonist, game)
        
        # Dodge but stay close (for excitement)
        avg_threat_x = sum(t[0] for t in threats) / len(threats)
        avg_threat_y = sum(t[1] for t in threats) / len(threats)
        
        # Move perpendicular to threat (creates near-miss)
        dx = protagonist.head.x - avg_threat_x
        dy = protagonist.head.y - avg_threat_y
        
        # Perpendicular direction
        perp_x, perp_y = -dy, dx
        target_x = protagonist.head.x + perp_x * 3
        target_y = protagonist.head.y + perp_y * 3
        
        return Director._move_toward(protagonist, target_x, target_y)
    
    @staticmethod
    def leader_dethrone(protagonist: Train, game) -> int:
        """Hunt the current leader specifically"""
        leader = max((t for t in game.trains if t.alive and t.id != protagonist.id), 
                    key=lambda t: t.length, default=None)
        if not leader:
            return 1
        return Director._move_toward(protagonist, leader.head.x, leader.head.y)
    
    @staticmethod
    def _move_toward(train: Train, target_x: int, target_y: int) -> int:
        """Return turn decision to move toward target"""
        dx = target_x - train.head.x
        dy = target_y - train.head.y
        
        # Account for wrapping
        if abs(dx) > WORLD_W // 2:
            dx = -dx
        if abs(dy) > WORLD_H // 2:
            dy = -dy
        
        # Current direction
        cur_dx, cur_dy = train.direction
        
        # Calculate angle to target
        target_angle = math.atan2(dy, dx)
        current_angle = math.atan2(cur_dy, cur_dx)
        
        diff = target_angle - current_angle
        while diff > math.pi: diff -= 2 * math.pi
        while diff < -math.pi: diff += 2 * math.pi
        
        if abs(diff) < 0.3:
            return 1  # Straight
        elif diff > 0:
            return 2  # Right
        else:
            return 0  # Left

# Scenario presets
SCENARIOS = {
    'underdog_comeback': {
        'name': 'Underdog Comeback',
        'description': 'Small train defeats giants',
        'script': Director.underdog_comeback,
        'protagonist_size': 4,  # Start small
        'enemy_sizes': [12, 15, 18, 10, 8],  # Big enemies
    },
    'kill_rampage': {
        'name': 'Kill Rampage',
        'description': 'Aggressive hunting spree',
        'script': Director.kill_rampage,
        'protagonist_size': 10,
        'enemy_sizes': [6, 7, 8, 5, 6, 7],
    },
    'narrow_escape': {
        'name': 'Narrow Escape',
        'description': 'Dodging death repeatedly',
        'script': Director.narrow_escape,
        'protagonist_size': 8,
        'enemy_sizes': [10, 12, 9, 11, 8],
    },
    'leader_dethrone': {
        'name': 'Dethrone the King',
        'description': 'Take down the leader',
        'script': Director.leader_dethrone,
        'protagonist_size': 8,
        'enemy_sizes': [20, 6, 7, 5, 6],  # One big leader
    },
}

In [ ]:
# === HEADLESS GAME ===
class HeadlessGame:
    def __init__(self, num_trains: int = 10, scenario: Optional[Dict] = None):
        self.trains: List[Train] = []
        self.food: List[Tuple[int, int]] = []
        self.particles: List[Dict] = []
        self.bullets: List[Dict] = []
        self.frame = 0
        self.events: List[Dict] = []  # For excitement model
        self.excitement_model = ExcitementModel()
        self.protagonist: Optional[Train] = None
        
        names = ['Shadow', 'Wolf', 'Blaze', 'Ice', 'Thunder', 'Phoenix', 'Storm', 'Ninja', 'Ghost', 'Viper']
        
        if scenario:
            # Director mode: set up scripted scenario
            self._setup_scenario(scenario, names)
        else:
            # Discovery mode: random setup
            for i in range(num_trains):
                x, y = random.randint(15, WORLD_W-15), random.randint(15, WORLD_H-15)
                train = Train(
                    id=i, name=names[i % len(names)] + str(random.randint(10, 99)),
                    segments=[Segment(x-j, y, gun=random.choice(GUN_TYPES) if j > 0 and random.random() < 0.4 else None) for j in range(5)],
                    color=TRAIN_COLORS[i % len(TRAIN_COLORS)],
                    direction=random.choice([(1,0), (-1,0), (0,1), (0,-1)])
                )
                self.trains.append(train)
        
        for _ in range(50):
            self.food.append((random.randint(0, WORLD_W-1), random.randint(0, WORLD_H-1)))
    
    def _setup_scenario(self, scenario: Dict, names: List[str]):
        """Set up a directed scenario"""
        # Create protagonist
        x, y = WORLD_W // 2, WORLD_H // 2
        size = scenario['protagonist_size']
        self.protagonist = Train(
            id=0, name='★' + names[0] + '★',
            segments=[Segment(x-j, y, gun=random.choice(GUN_TYPES) if j > 1 else None) for j in range(size)],
            color=(0, 255, 100),  # Hero green
            direction=(1, 0),
            is_protagonist=True,
            script=scenario['script']
        )
        self.trains.append(self.protagonist)
        
        # Create enemies
        for i, enemy_size in enumerate(scenario['enemy_sizes']):
            angle = (i / len(scenario['enemy_sizes'])) * 2 * math.pi
            dist = 25 + random.randint(-5, 5)
            x = int(WORLD_W // 2 + math.cos(angle) * dist) % WORLD_W
            y = int(WORLD_H // 2 + math.sin(angle) * dist) % WORLD_H
            
            train = Train(
                id=i+1, name=names[(i+1) % len(names)] + str(random.randint(10, 99)),
                segments=[Segment(x-j, y, gun=random.choice(GUN_TYPES) if j > 0 else None) for j in range(enemy_size)],
                color=TRAIN_COLORS[(i+1) % len(TRAIN_COLORS)],
                direction=random.choice([(1,0), (-1,0), (0,1), (0,-1)])
            )
            self.trains.append(train)
    
    def step(self):
        self.frame += 1
        alive = [t for t in self.trains if t.alive]
        
        # Track game state for excitement model
        game_state = {
            'trains_nearby': sum(1 for t in alive for t2 in alive if t.id != t2.id and 
                                abs(t.head.x - t2.head.x) + abs(t.head.y - t2.head.y) < 15),
            'leader': max(alive, key=lambda t: t.length) if alive else None,
        }
        
        for train in alive:
            train.turn(train.think(self))
            train.move()
        
        # Food
        for train in alive:
            for i, (fx, fy) in enumerate(self.food):
                if train.head.x == fx and train.head.y == fy:
                    train.grow(2)
                    self.food.pop(i)
                    self.food.append((random.randint(0, WORLD_W-1), random.randint(0, WORLD_H-1)))
                    break
        
        # Collisions
        for train in alive:
            for other in alive:
                if other.id == train.id: continue
                for seg in other.segments:
                    if train.head.x == seg.x and train.head.y == seg.y:
                        if train.length > other.length:
                            self._handle_kill(train, other, game_state)
                        elif other.length > train.length:
                            self._handle_kill(other, train, game_state)
                        else:
                            train.alive = other.alive = False
                            self._add_particles(train)
                            self._add_particles(other)
                        break
                if not train.alive: break
        
        # Respawn (not protagonist in director mode)
        for train in self.trains:
            if not train.alive and not train.is_protagonist:
                self._respawn(train)
        
        # Update particles
        self.particles = [p for p in self.particles if p['life'] > 0]
        for p in self.particles:
            p['x'] += p['vx']; p['y'] += p['vy']; p['vy'] += 0.2; p['life'] -= 1
    
    def _handle_kill(self, killer: Train, victim: Train, game_state: Dict):
        """Process a kill and record for excitement model"""
        killer.kills += 1
        killer.grow(victim.length // 2)
        self._add_particles(victim)
        victim.alive = False
        
        # Record event for excitement model
        event = {
            'frame': self.frame,
            'kill': 1,
            'streak': killer.kills,
            'length_diff': killer.length - victim.length,
            'velocity': len([d for d in killer.direction_history[-10:] if d == killer.direction]) if killer.direction_history else 0,
            'near_miss_count': 0,  # Could track this
            'trains_nearby': game_state['trains_nearby'],
            'killer_health_pct': killer.head.hp / killer.head.max_hp if killer.head else 1,
            'victim_length': victim.length,
            'leader_involved': 1 if (game_state['leader'] and (killer.id == game_state['leader'].id or victim.id == game_state['leader'].id)) else 0,
            'comeback': 1 if killer.length < victim.length else 0,
            'killer_id': killer.id,
            'killer_name': killer.name,
            'victim_name': victim.name,
            'is_strafing': killer.is_strafing(),
        }
        
        # Auto-label and add to training data
        features = self.excitement_model.extract_features(event, game_state)
        label = self.excitement_model.auto_label(event)
        self.excitement_model.add_sample(features, label)
        
        event['excitement_score'] = label
        event['features'] = features
        self.events.append(event)
        
        emoji = '🔥' if event['streak'] > 1 else '💀'
        print(f"  {emoji} {killer.name} → {victim.name} (excitement: {label:.2f})")
    
    def _add_particles(self, train: Train):
        for seg in train.segments:
            for _ in range(5):
                self.particles.append({
                    'x': seg.x * GRID + GRID//2, 'y': seg.y * GRID + GRID//2,
                    'vx': random.uniform(-4, 4), 'vy': random.uniform(-6, 2),
                    'life': random.randint(15, 30), 'color': train.color, 'size': random.randint(2, 5)
                })
    
    def _respawn(self, train: Train):
        x, y = random.randint(15, WORLD_W-15), random.randint(15, WORLD_H-15)
        train.segments = [Segment(x-j, y) for j in range(4)]
        train.direction = random.choice([(1,0), (-1,0), (0,1), (0,-1)])
        train.alive = True
        train.direction_history.clear()
        train.position_history.clear()

In [ ]:
# === RENDERER ===
class GameRenderer:
    def dim(self, c, f=0.5): return tuple(int(x*f) for x in c)
    
    def render(self, game: HeadlessGame, camera_target: Optional[Train] = None) -> Image.Image:
        img = Image.new('RGB', (CANVAS_W, CANVAS_H), (17, 17, 17))
        draw = ImageDraw.Draw(img)
        
        cam_x = (camera_target.head.x * GRID - CANVAS_W // 2) if camera_target and camera_target.head else (WORLD_W * GRID - CANVAS_W) // 2
        cam_y = (camera_target.head.y * GRID - CANVAS_H // 2) if camera_target and camera_target.head else (WORLD_H * GRID - CANVAS_H) // 2
        
        # Grid
        for x in range(0, WORLD_W * GRID, GRID):
            sx = x - cam_x
            if -GRID <= sx < CANVAS_W + GRID:
                draw.line([(sx, 0), (sx, CANVAS_H)], fill=(30, 35, 30))
        for y in range(0, WORLD_H * GRID, GRID):
            sy = y - cam_y
            if -GRID <= sy < CANVAS_H + GRID:
                draw.line([(0, sy), (CANVAS_W, sy)], fill=(30, 35, 30))
        
        # Food
        for fx, fy in game.food:
            sx, sy = fx * GRID - cam_x + GRID//2, fy * GRID - cam_y + GRID//2
            if -10 <= sx < CANVAS_W+10 and -10 <= sy < CANVAS_H+10:
                draw.ellipse([sx-4, sy-4, sx+4, sy+4], fill=(255, 255, 0))
        
        # Trains
        for train in sorted(game.trains, key=lambda t: t.length):
            if not train.alive: continue
            for i, seg in enumerate(train.segments):
                sx, sy = seg.x * GRID - cam_x, seg.y * GRID - cam_y
                if -GRID*2 <= sx < CANVAS_W+GRID*2 and -GRID*2 <= sy < CANVAS_H+GRID*2:
                    is_head = (i == 0)
                    color = train.color if is_head else self.dim(train.color, 0.6)
                    size = GRID - 2 if is_head else GRID - 4
                    off = (GRID - size) // 2
                    
                    if is_head and train.is_protagonist:
                        # Glow for protagonist
                        draw.ellipse([sx-3, sy-3, sx+GRID+3, sy+GRID+3], fill=self.dim(train.color, 0.3))
                    
                    draw.rectangle([sx+off, sy+off, sx+off+size, sy+off+size], fill=color)
                    
                    if seg.gun and not is_head:
                        cx, cy = sx + GRID//2, sy + GRID//2
                        draw.ellipse([cx-3, cy-3, cx+3, cy+3], fill=seg.gun['color'])
                    
                    if is_head:
                        hp = seg.hp / seg.max_hp
                        draw.rectangle([sx+1, sy-4, sx+GRID-1, sy-2], fill=(50,0,0))
                        draw.rectangle([sx+1, sy-4, sx+1+int((GRID-2)*hp), sy-2], fill=(0,255,0))
        
        # Particles
        for p in game.particles:
            px, py = p['x'] - cam_x, p['y'] - cam_y
            if 0 <= px < CANVAS_W and 0 <= py < CANVAS_H:
                alpha = p['life'] / 30
                draw.ellipse([px-p['size'], py-p['size'], px+p['size'], py+p['size']], 
                           fill=tuple(int(c*alpha) for c in p['color']))
        
        # Leaderboard
        top = sorted(game.trains, key=lambda t: t.length, reverse=True)[:5]
        draw.rectangle([CANVAS_W-140, 5, CANVAS_W-5, 100], fill=(0,0,0), outline=(0,100,0))
        for i, t in enumerate(top):
            prefix = '👑' if i == 0 else f'{i+1}.'
            draw.text((CANVAS_W-135, 10+i*18), f"{prefix} {t.name[:7]}: {t.length}", fill=t.color)
        
        return img
    
    def save_clip(self, frames: List[Image.Image], filename: str):
        with imageio.get_writer(filename, fps=FPS, quality=8, macro_block_size=1) as w:
            for f in frames:
                w.append_data(np.array(f))
        print(f"  ✅ {filename}")

In [ ]:
# === CLIP EXTRACTOR ===
class ClipExtractor:
    def __init__(self, game: HeadlessGame, renderer: GameRenderer):
        self.game = game
        self.renderer = renderer
    
    def run(self, frames: int = 4500) -> Dict[int, Image.Image]:
        print(f"🎮 Running {frames/FPS:.0f}s simulation...\n")
        all_frames = {}
        
        for i in range(frames):
            self.game.step()
            target = self.game.protagonist or max((t for t in self.game.trains if t.alive), key=lambda t: t.length, default=None)
            all_frames[self.game.frame] = self.renderer.render(self.game, target)
            
            if i > 0 and i % 900 == 0:
                print(f"  {i/FPS:.0f}s - {len(self.game.events)} events")
        
        # Train excitement model
        print(f"\n🧠 Training excitement model...")
        self.game.excitement_model.train()
        
        # Re-score events with trained model
        for event in self.game.events:
            event['learned_score'] = self.game.excitement_model.predict(event['features'])
        
        return all_frames
    
    def extract(self, all_frames: Dict, n: int = 5) -> List[Dict]:
        # Filter: must be strafing and have good score
        valid = [e for e in self.game.events if e.get('is_strafing', True)]
        best = sorted(valid, key=lambda e: e.get('learned_score', e['excitement_score']), reverse=True)[:n]
        
        print(f"\n🎬 Top {len(best)} clips:")
        clips = []
        for i, e in enumerate(best):
            start = max(1, e['frame'] - FPS * 3)
            end = min(max(all_frames.keys()), e['frame'] + FPS * 4)
            clip_frames = [all_frames[f] for f in range(start, end) if f in all_frames]
            if len(clip_frames) > FPS * 2:
                score = e.get('learned_score', e['excitement_score'])
                print(f"  #{i+1} {e['killer_name']} → {e['victim_name']} (score: {score:.2f})")
                clips.append({'event': e, 'frames': clip_frames, 'rank': i+1})
        return clips
    
    def save(self, clips: List[Dict], outdir: str = 'clips') -> List[str]:
        os.makedirs(outdir, exist_ok=True)
        print(f"\n💾 Saving {len(clips)} clips...")
        files = []
        for c in clips:
            score = c['event'].get('learned_score', c['event']['excitement_score'])
            fn = f"{outdir}/clip_{c['rank']:02d}_{int(score*100)}pts.mp4"
            self.renderer.save_clip(c['frames'], fn)
            files.append(fn)
        return files

---
# 🎬 MODE 1: Discovery (ML learns excitement)

In [ ]:
print("🔍 DISCOVERY MODE")
print("=" * 50)
print("ML model learns what makes moments exciting\n")

game = HeadlessGame(num_trains=10)
renderer = GameRenderer()
extractor = ClipExtractor(game, renderer)

all_frames = extractor.run(frames=FPS * 180)  # 3 minutes
clips = extractor.extract(all_frames, n=5)
discovery_files = extractor.save(clips, 'discovery_clips')

print(f"\n🎉 {len(discovery_files)} discovery clips saved!")

In [ ]:
# Preview discovery clips
for f in discovery_files[:2]:
    print(f"\n{f}")
    display(Video(f, embed=True, width=640))

---
# 🎬 MODE 2: Director (Scripted Scenarios)

In [ ]:
print("🎬 DIRECTOR MODE")
print("=" * 50)
print("Available scenarios:")
for key, s in SCENARIOS.items():
    print(f"  • {key}: {s['description']}")

In [ ]:
# Run all scenarios
director_files = []

for scenario_key, scenario in SCENARIOS.items():
    print(f"\n🎬 Filming: {scenario['name']}")
    print("-" * 40)
    
    game = HeadlessGame(scenario=scenario)
    renderer = GameRenderer()
    
    frames = []
    for i in range(FPS * 30):  # 30 sec per scenario
        game.step()
        if game.protagonist and not game.protagonist.alive:
            print(f"  Protagonist died at {i/FPS:.1f}s")
            break
        frames.append(renderer.render(game, game.protagonist))
    
    if len(frames) > FPS * 5:  # At least 5 seconds
        fn = f"director_clips/{scenario_key}.mp4"
        os.makedirs('director_clips', exist_ok=True)
        renderer.save_clip(frames, fn)
        director_files.append(fn)

print(f"\n🎉 {len(director_files)} director clips saved!")

In [ ]:
# Preview director clips
for f in director_files[:2]:
    print(f"\n{f}")
    display(Video(f, embed=True, width=640))

---
# 📦 Download All Clips

In [ ]:
from google.colab import files
import zipfile

all_clips = discovery_files + director_files
with zipfile.ZipFile('locomotio_promo_clips.zip', 'w') as z:
    for f in all_clips:
        z.write(f)

print(f"📦 Downloading {len(all_clips)} clips...")
files.download('locomotio_promo_clips.zip')